In [6]:
from PIL import Image
import numpy as np
import cv2
from mtcnn import MTCNN
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import transforms, models
from IPython.display import display, clear_output
import matplotlib.pyplot as plt


2025-05-07 15:12:21.911632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import torch
import torch.nn as nn
from torchvision import models
from collections import OrderedDict

# Define the same model architecture used during training
class FaceMaskModel(nn.Module):
    def __init__(self):
        super(FaceMaskModel, self).__init__()
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 2)  # Binary classification

    def forward(self, x):
        return self.model(x)

# Instantiate model
model = FaceMaskModel()

# Load the saved state_dict
state_dict = torch.load("FaceMaskModel.pt", map_location=torch.device("cpu"))

# Remove 'module.' prefix if model was saved with DataParallel
if list(state_dict.keys())[0].startswith("module."):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        new_key = k.replace("module.", "")
        new_state_dict[new_key] = v
    model.load_state_dict(new_state_dict)
else:
    model.load_state_dict(state_dict)

# Set model to evaluation mode
model.eval()

print("✅ Model loaded successfully and ready for inference.")





✅ Model loaded successfully and ready for inference.


In [7]:
# --- Preprocessing ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])
labels = ['Without Mask', 'With Mask']
detector = MTCNN()


In [8]:
# --- Video Stream Setup ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Camera could not be opened")
    exit()

print("📹 Press 'q' to stop the video stream")


[ WARN:0@34.749] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


📹 Press 'q' to stop the video stream


In [9]:
# 8. Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to capture frame")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes = detector.detect_faces(rgb_frame)

    for box in boxes:
        x, y, width, height = box['box']
        x, y = max(0, x), max(0, y)
        face = frame[y:y+height, x:x+width]

        if face.size == 0:
            continue

        pil_face = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        transformed_face = transform(pil_face).unsqueeze(0)

        with torch.no_grad():
            outputs = model(transformed_face)
            _, predicted = torch.max(outputs, 1)

        label = labels[predicted.item()]
        color = (0, 255, 0) if predicted.item() == 1 else (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+width, y+height), color, 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Mask Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()